## Frozen Soil Model 
### Using Freezing-point depression model (temperature and water content dependent) for phase partitioning
### Using Crank-Nicolson scheme to solve Diffusion Equation

#### Algorithm 
* Get initial Soil Temperature and Soil Moisture (Total and liquid moisture content) profiles
* Call PhaseChange method, using freezing-point depression equation, to partitioin total water content into liquid and ice
* Soil heat capacity is updated inside PhaseChange using current timestep liquid/ice contents
* Thermal conductivities are computed at the end of the timestep for the next timestep

#### Problem description

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter 
import pandas as pd
import imageio, re
import json, os
import frozensoil as FS


### Create an object/instance of the model, a configuration file is provided which is later used in the initialized step to initialize the state of the model at time t = 0

In [2]:
fs = FS.FrozenSoil('./configs/config.json')

### Initialize the state of the model at the initial time, sets all the variables needed for phase partition and heat equation

In [3]:
fs.initialize()

### All set for running the model and advancing the timestep. The forcing data (time series), if provided, will be used and the corresponding variables will be updated each time step at the model advancement stage

In [4]:
#set top surface temperature, if boundary condition is not set here, the default is -12 C
cycles = fs.get_NTsteps()
bc = np.ones(cycles)*260
for i,b in enumerate(bc):
    if i > 3600:
        bc[i] = 285
fs.set_surface_bc(bc)

In [5]:
#fs.run_model()
fs.run_model_forcing()

### DONE, and data ready for post-processing. Here, the BMI solution of the frozen soil model is used as an example. 
### Solution format: 0: time [s], 1: soil temperature [K], 2: liquid water [-], 3: total soil moisture [-]

In [ ]:
solution = fs.get_solution()

In [ ]:
solution[1]

### Postprocessing (Visualization, comparison etc.) 
### Get animated figure of freeze-thaw simulation

In [ ]:
# Example: Comparison against frozen soil run through BMI

# user defined path
out_dir = '/Users/ahmadjan/Core/SimulationsData/postprocessing/frozensoil/'

#BMI simulated data
bmi_datafile='/Users/ahmadjan/codes/bmi/bmi-frozensoil/_build/testing/bmifrozensoilcxx.out'

out_path= os.path.join(out_dir,'comp_tmp1') #processed data (images, movie clip) will be stored here
out_path

In [ ]:

Z = fs.get_Z()
A = solution
cycles

In [ ]:
# Read BMI simulation data
BMI_ST = []
BMI_Water =[]
BMI_Time = []
BMI_Ice = []
nz = len(Z)
count = 0
with open(bmi_datafile,'r') as file:
    for f in file.readlines()[9:]:
        sp = f.split()
        if "Time" in f:
            BMI_Time.append(float(sp[2]))
        elif ('Final' not in f and len(sp) == 3):
            BMI_ST.append(float(sp[0][:-1]))
            BMI_Water.append(float(sp[1][:-1]))
            BMI_Ice.append(float(sp[2]))
            count = count + 1

bmiST = np.reshape(np.array(BMI_ST), (-1,nz))
bmiW = np.reshape(np.array(BMI_Water), (-1,nz))
bmiIce = np.reshape(np.array(BMI_Ice), (-1,nz))


In [ ]:
for i in range(0,cycles+1,12):
    fig, axs = plt.subplots(1,2,figsize=(10,6))
    #
    axs[0].plot(bmiST[i],Z,color='r',label='Soil T (BMI)')
    axs[0].plot(A[i][1],Z,color='k',linestyle='dashed',label='Soil T (Python)')
    axs[0].vlines(bmiST[0],ymin=0,ymax=7,color='grey',label='Initial ST',linestyle='dotted')
    axs[0].set_xlabel('Soil temperature [K]',fontsize=12)
    
    axs[1].plot(bmiW[i],Z,color='b',label='Liquid')
    axs[1].plot(bmiIce[i],Z,color='g',label='Ice')
    axs[1].plot(A[i][2],Z,color='k',linestyle='dashed',label='Liquid (Python)')
    axs[1].plot(bmiW[0],Z,color='grey',label='Initial SMC',linestyle='dotted')
    
    axs[1].set_xlabel('Soil moisture [K]',fontsize=12)
    axs[0].text(262.5, 5., 'Time = %s [d]'%(BMI_Time[i]/(3600.*24)), fontsize=12, color="green")

    axs[0].set_xlim(260,285.1)
    axs[1].set_xlim(-0.01,0.5)
    
    for j in range(2):
        axs[j].set_ylim(0.,6.9)
        axs[j].invert_yaxis()
        axs[j].xaxis.set_tick_params(labelsize=12)
        axs[j].yaxis.set_tick_params(labelsize=12)
        axs[0].set_ylabel('Depth [m]',fontsize=12)
        axs[0].legend(loc = 'lower left',labelspacing=.2,fontsize=10)
        axs[1].legend(loc = 'lower center',labelspacing=.1,fontsize=10)
    plt.savefig(out_path + '/fig-%s.png'%(BMI_Time[i]/3600.),dpi=100)
    plt.close()

In [ ]:

images = []
def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)
print(out_path)
dir_files = os.listdir(out_path)

Files = sorted_nicely(dir_files)
Files = [f for f in Files if f.endswith('.png')]
print (len(dir_files), Files[:3])

for filename in Files:
    images.append(imageio.imread(os.path.join(out_path,filename)))
imageio.mimsave(out_path+'/sim_clip.gif', images)